**Add CUDA deps**

note: change runtime to T4 GPU

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-txyqc28k
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-txyqc28k
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4294 sha256=09417ef5ae7da46a138413e80d62e9c2b86ab2496a58fdc3350b40ab472db97a
  Stored in directory: /tmp/pip-ephem-wheel-cache-a177qw1r/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [3]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [4]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 12.5 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2023.1-cp310-cp310-linux_x86_64.whl size=661263 sha256=f5e83d8e4a4610474dec2091724b0a72595fd90de039e19fd68aab20d169a074
  Stored in directory: /root/.cache/pip/wheels/46/65/06/b997165edd2fd9690c3497ca54ea4485b571d7bd959c21c6c4
Successfully built pycuda


In [6]:
# https://vitalitylearning.medium.com/running-cuda-in-google-colab-525a92efcf75
import pycuda.driver as drv
import pycuda.autoinit
drv.init()
print("%d device(s) found." % drv.Device.count())
for i in range(drv.Device.count()):
  dev = drv.Device(i)
  print("Device #%d: %s" % (i, dev.name()))
  print(" Compute Capability: %d.%d" % dev.compute_capability())
  print(" Total Memory: %s GB" % (dev.total_memory() // (1024 * 1024 * 1024)))

1 device(s) found.
Device #0: Tesla T4
 Compute Capability: 7.5
 Total Memory: 14 GB


In [7]:
 !cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0xffffffff
cpu MHz		: 2299.998
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs mmio_stale_data retbleed
bogomips	: 4599.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtua

In [8]:
!cat /proc/meminfo

MemTotal:       13294208 kB
MemFree:         9618716 kB
MemAvailable:   12187104 kB
Buffers:           74144 kB
Cached:          2703680 kB
SwapCached:            0 kB
Active:           792292 kB
Inactive:        2472728 kB
Active(anon):       3172 kB
Inactive(anon):   498008 kB
Active(file):     789120 kB
Inactive(file):  1974720 kB
Unevictable:           4 kB
Mlocked:               4 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:              6164 kB
Writeback:             0 kB
AnonPages:        487204 kB
Mapped:           363480 kB
Shmem:             13980 kB
KReclaimable:      91004 kB
Slab:             135396 kB
SReclaimable:      91004 kB
SUnreclaim:        44392 kB
KernelStack:        4752 kB
PageTables:         8436 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6647104 kB
Committed_AS:    2058080 kB
VmallocTotal:   34359738367 kB
VmallocUsed:       77552 kB
VmallocChunk:          0 kB
Percpu:          

**CUDA LLP**

In [9]:
%%cuda --name cuda_SSSP_LLP.cu


#include <stdlib.h>
#include <stdio.h>
#include <sys/time.h>

/* power of 2 */
#define N 32768


#define NUM_PRED 3

#define BLOCKSIZE 128

/* N / BLOCKSIZE */
#define GRIDSIZE 256

#define MAXWEIGHT 3

/*
 * Compute whether there exists the number 1 in the fixed array.
 *
 * ret: must have *ret==0 before calling the function. After execution, *ret will be 1
 *      if there is a 0 in the fixed array, else it will be 0.
 * fixed: array of 1s and 0s, of length N
 */
__global__ void hasUnfixed(int *ret, int *fixed) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx >= N) {
        return;
    }
    if (fixed[idx] == 0) {
        *ret = 1;
    }
}

/*
 * Execute a thread of LLP for Shortest Path (single source).
 *
 * N: number of vertices
 * predecessors: predecessors list for each node. predecessors[k][i][0] is the kth
 *     predecessor of the ith node, predecessors[k][i][1] is the corresponding weight.
 *     The number predecessors[k][i][0] should be -1 if k is greater than or equal to
 *     the total number of incoming edges for the ith node.
 * maxNumPred: the maximum number of incoming edges for a single node, taken across
 *     all nodes.
 * latticeValues: lattice values that may be read and written by threads
 * fixed: the ith value is 0 if the ith node is not fixed, else it is 1
 * vals: the values from the minimization problem in SSSP LLP
 * beta: the minimum of the vals vector
 *
 */
__global__ void llpThreadKernel(int (*predecessors) [N][2], int maxNumPred, int * in_latticeValues, int * out_latticeValues, int * fixed, int * vals, int beta) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;

    if (idx < N) {
        out_latticeValues[idx] = in_latticeValues[idx];
        if (fixed[idx]) {
            return;
        }

        if (vals[idx] == beta) {
            out_latticeValues[idx] = beta;
            fixed[idx] = 1;
        } else {
            for (int m = 0; m < maxNumPred; ++m) {
                int i = predecessors[m][idx][0];
                int weight = predecessors[m][idx][1];
                if (i == -1) {
                    break;
                }
                if (fixed[i] && (in_latticeValues[idx] >= in_latticeValues[i] + weight)) {
                    fixed[idx] = 1;
                }
            }

            if (fixed[idx]) {
                return;
            }

            int newMin = -1;
            for (int m = 0; m < maxNumPred; ++m) {
                int i = predecessors[m][idx][0];
                int weight = predecessors[m][idx][1];
                if (i == -1) {
                    break;
                }
                int candidate = in_latticeValues[i] + weight;
                if (newMin == -1 || (candidate < newMin)) {
                    newMin = candidate;
                }
            }

            out_latticeValues[idx] = max(beta, newMin);
        }
    }
}

__global__ void llpFindMin(int (*predecessors) [N][2], int maxNumPred, int *latticeValues, int *fixed, int *vals, int *nonempty) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;

    if (idx < N) {
        int minVal = -1;
        int found = 0;
        for (int i = 0; i < maxNumPred; i += 1) {
            if (predecessors[i][idx][0] == -1) {
                break;
            }
            int pred = fixed[predecessors[i][idx][0]] && !(fixed[idx]);
            if (pred) {
                found = 1;
            } else {
                continue;
            }
            int candidate = latticeValues[predecessors[i][idx][0]] + predecessors[i][idx][1];
            if (minVal == -1 || (candidate < minVal)) {
                minVal = candidate;
            }
        }
        vals[idx] = minVal;
        if (found) {
            *nonempty = 1;
        }
    }
}

__global__ void minReduce(int *in_vals, int *out_vals, int s) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;

    if (idx % (2*s) != 0) {
        return;
    }

    if (in_vals[idx] == -1) {
        out_vals[idx] = in_vals[idx+s];
    } else if (in_vals[idx+s] == -1) {
        out_vals[idx] = in_vals[idx];
    } else {
        out_vals[idx] = min(in_vals[idx], in_vals[idx+s]);
    }
}


int main() {
    int error = 0;

    /* https://stackoverflow.com/questions/2150291/how-do-i-measure-a-time-interval-in-c */
    struct timeval t1, t2;
    double elapsedTime;

    /* seed -- create random graph */
    srand(128);

    int (*h_predecessors)[N][2];
    int predBytes = NUM_PRED * sizeof *h_predecessors;
    h_predecessors = (int (*)[N][2])malloc(predBytes);

    /* initialize random graph (plus a linear path beginning to end) */
    for (int i = 0; i < N; ++i) {
        h_predecessors[0][i][0] = i-1;
        h_predecessors[0][i][1] = 1;


        int j = 1;
        while (j < min(N-1,NUM_PRED)) {
            int pred = rand() % N;
            int weight = (MAXWEIGHT > 1) ? (1 + (rand() % (MAXWEIGHT - 1))) : 1;

            // no self-loop
            if (pred == i) {
                continue;
            }

            // check it was not already used
            int found = 0;
            for (int k = 0; k < j; ++k) {
                if (h_predecessors[k][i][0] == pred) {
                    found = 1;
                }
            }
            if (found) {
                continue;
            }

            h_predecessors[j][i][0] = pred;
            h_predecessors[j][i][1] = weight;
            ++j;
        }

        // sentinel
        if (j < NUM_PRED) {
            h_predecessors[j][i][0] = -1;
        }
    }


    int (*d_predecessors)[N][2];
    cudaMalloc((int (**) [N][2])&d_predecessors, predBytes);
    cudaMemcpy(d_predecessors, h_predecessors, predBytes, cudaMemcpyHostToDevice);


    int *h_latticeValues;
    int *h_latticeValues_2;
    int bytes = N * sizeof *h_latticeValues;
    h_latticeValues = (int*)malloc(bytes);
    h_latticeValues_2 = (int*)malloc(bytes);
    for (int i = 0; i < N; ++i) {
        h_latticeValues[i] = 0;
        h_latticeValues_2[i] = 0;
    }

    int *d_latticeValues;
    int *d_latticeValues_2;
    cudaMalloc((int**)&d_latticeValues, bytes);
    cudaMalloc((int**)&d_latticeValues_2, bytes);
    cudaMemcpy(d_latticeValues, h_latticeValues, bytes, cudaMemcpyHostToDevice);
    cudaMemcpy(d_latticeValues_2, h_latticeValues_2, bytes, cudaMemcpyHostToDevice);

    int *h_fixed;
    h_fixed = (int*)malloc(bytes);
    h_fixed[0] = 1;
    for (int i = 1; i < N; ++i) {
        h_fixed[i] = 0;
    }

    int *d_fixed;
    cudaMalloc((int **)&d_fixed, bytes);
    cudaMemcpy(d_fixed, h_fixed, bytes, cudaMemcpyHostToDevice);

    int *h_vals;
    int *h_vals_2;
    h_vals = (int*)malloc(bytes);
    h_vals_2 = (int*)malloc(bytes);
    for (int i = 0; i < N; ++i) {
        h_vals[i] = 0;
        h_vals_2[i] = 0;
    }

    int *d_vals;
    int *d_vals_2;
    cudaMalloc((int **)&d_vals, bytes);
    cudaMalloc((int **)&d_vals_2, bytes);
    cudaMemcpy(d_vals, h_vals, bytes, cudaMemcpyHostToDevice);
    cudaMemcpy(d_vals_2, h_vals_2, bytes, cudaMemcpyHostToDevice);

    int *h_ret;
    int *h_nonempty;
    h_ret = (int*)malloc(1);
    *h_ret = 0;
    h_nonempty = (int*)malloc(1);
    *h_nonempty = 0;

    int *d_ret;
    int *d_nonempty;
    cudaMalloc((int **)&d_ret, 1);
    cudaMalloc((int **)&d_nonempty, 1);
    cudaMemcpy(d_ret, h_ret, 1, cudaMemcpyHostToDevice);
    cudaMemcpy(d_nonempty, h_nonempty, 1, cudaMemcpyHostToDevice);


    int done = 0;

    gettimeofday(&t1, NULL);

    while(1) {
        *h_ret = 0;
        *h_nonempty = 0;
        cudaMemcpy(d_ret, h_ret, 1, cudaMemcpyHostToDevice);
        cudaMemcpy(d_nonempty, h_nonempty, 1, cudaMemcpyHostToDevice);

        hasUnfixed<<<GRIDSIZE, BLOCKSIZE>>>(d_ret, d_fixed);
        cudaDeviceSynchronize();
        cudaMemcpy(h_ret, d_ret, 1, cudaMemcpyDeviceToHost);
        if (*h_ret == 0) {
            done = 1;
            break; /* we are done */
        }

        llpFindMin<<<GRIDSIZE, BLOCKSIZE>>>(d_predecessors, NUM_PRED, d_latticeValues, d_fixed, d_vals, d_nonempty);
        cudaDeviceSynchronize();
        cudaMemcpy(h_nonempty, d_nonempty, 1, cudaMemcpyDeviceToHost);
        if (*h_nonempty == 0) {
            printf("------ ERROR: non-fixed nodes not reachable ------");
            error = 1;
            break;
        }

        cudaMemcpy(h_vals, d_vals, bytes, cudaMemcpyDeviceToHost);
        memcpy(h_vals_2, h_vals, bytes);
        cudaMemcpy(d_vals_2, h_vals_2, bytes, cudaMemcpyHostToDevice);

        for (int s = 1; s < N; s *= 2) {
            minReduce<<<GRIDSIZE, BLOCKSIZE>>>(d_vals_2, d_vals_2, s);
            cudaDeviceSynchronize();
        }

        cudaMemcpy(h_vals_2, d_vals_2, bytes, cudaMemcpyDeviceToHost);
        int beta = h_vals_2[0];

        llpThreadKernel<<<GRIDSIZE, BLOCKSIZE>>>(d_predecessors, NUM_PRED, d_latticeValues, d_latticeValues_2, d_fixed, d_vals, beta);
        cudaDeviceSynchronize();
        cudaMemcpy(h_latticeValues_2, d_latticeValues_2, bytes, cudaMemcpyDeviceToHost);
        memcpy(h_latticeValues, h_latticeValues_2, bytes);
        cudaMemcpy(d_latticeValues, h_latticeValues, bytes, cudaMemcpyHostToDevice);
    }

    gettimeofday(&t2, NULL);

    printf("\n");

    if (done && !error) {
        printf("DONE\n");
        printf("------ BEGIN VALUES ------\n");
        for (int i = 0; i < N; ++i) {
            printf("%d: %d\n", i, h_latticeValues[i]);
        }
        elapsedTime = (t2.tv_sec - t1.tv_sec) * 1000.0;
        elapsedTime += (t2.tv_usec - t1.tv_usec) / 1000.0;
        printf("------ END VALUES ------\n");
        printf("Time %f ms.\n", elapsedTime);
    } else {
        printf("ERROR\n");
    }

    free(h_predecessors);
    free(h_latticeValues);
    free(h_latticeValues_2);
    free(h_fixed);
    free(h_vals);
    free(h_vals_2);
    free(h_ret);
    free(h_nonempty);

    cudaFree(d_predecessors);
    cudaFree(d_latticeValues);
    cudaFree(d_latticeValues_2);
    cudaFree(d_fixed);
    cudaFree(d_vals);
    cudaFree(d_vals_2);
    cudaFree(d_ret);
    cudaFree(d_nonempty);
}


'File written in /content/src/cuda_SSSP_LLP.cu'

In [10]:
!nvcc -arch=sm_75 -o "/content/src/cuda_SSSP_LLP.o" /content/src/cuda_SSSP_LLP.cu

In [11]:
!chmod 755 /content/src/cuda_SSSP_LLP.o
!/content/src/cuda_SSSP_LLP.o

Streaming output truncated to the last 5000 lines.
27770: 13
27771: 13
27772: 14
27773: 14
27774: 15
27775: 14
27776: 14
27777: 7
27778: 8
27779: 9
27780: 10
27781: 11
27782: 12
27783: 13
27784: 12
27785: 11
27786: 12
27787: 13
27788: 11
27789: 12
27790: 13
27791: 13
27792: 10
27793: 11
27794: 12
27795: 13
27796: 13
27797: 12
27798: 13
27799: 14
27800: 14
27801: 10
27802: 10
27803: 10
27804: 11
27805: 12
27806: 13
27807: 12
27808: 13
27809: 14
27810: 12
27811: 10
27812: 11
27813: 8
27814: 9
27815: 10
27816: 11
27817: 9
27818: 10
27819: 11
27820: 12
27821: 7
27822: 8
27823: 9
27824: 10
27825: 11
27826: 12
27827: 13
27828: 13
27829: 12
27830: 10
27831: 11
27832: 10
27833: 8
27834: 9
27835: 10
27836: 10
27837: 11
27838: 12
27839: 12
27840: 13
27841: 14
27842: 14
27843: 14
27844: 12
27845: 13
27846: 11
27847: 12
27848: 13
27849: 12
27850: 13
27851: 12
27852: 13
27853: 13
27854: 14
27855: 8
27856: 9
27857: 10
27858: 11
27859: 12
27860: 13
27861: 12
27862: 10
27863: 11
27864: 12
27865: 13
27

**CPU**

In [12]:
%%cuda --name cpu_SSSP_LLP.cu


#include <stdlib.h>
#include <stdio.h>
#include <sys/time.h>

/* power of 2 */
#define N 32768


#define NUM_PRED 3

#define BLOCKSIZE 128

/* N / BLOCKSIZE */
#define GRIDSIZE 256

#define MAXWEIGHT 3

/*
 * Compute whether there exists the number 1 in the fixed array.
 *
 * ret: must have *ret==0 before calling the function. After execution, *ret will be 1
 *      if there is a 0 in the fixed array, else it will be 0.
 * fixed: array of 1s and 0s, of length N
 */
void hasUnfixed(int idx, int *ret, int *fixed) {
    if (idx >= N) {
        return;
    }
    if (fixed[idx] == 0) {
        *ret = 1;
    }
}

/*
 * Execute a thread of LLP for Shortest Path (single source).
 *
 * N: number of vertices
 * predecessors: predecessors list for each node. predecessors[k][i][0] is the kth
 *     predecessor of the ith node, predecessors[k][i][1] is the corresponding weight.
 *     The number predecessors[k][i][0] should be -1 if k is greater than or equal to
 *     the total number of incoming edges for the ith node.
 * maxNumPred: the maximum number of incoming edges for a single node, taken across
 *     all nodes.
 * latticeValues: lattice values that may be read and written by threads
 * fixed: the ith value is 0 if the ith node is not fixed, else it is 1
 * vals: the values from the minimization problem in SSSP LLP
 * beta: the minimum of the vals vector
 *
 */
void llpThreadKernel(int idx, int (*predecessors) [N][2], int maxNumPred, int * in_latticeValues, int * out_latticeValues, int * fixed, int * vals, int beta) {
    if (idx < N) {
        out_latticeValues[idx] = in_latticeValues[idx];
        if (fixed[idx]) {
            return;
        }

        if (vals[idx] == beta) {
            out_latticeValues[idx] = beta;
            fixed[idx] = 1;
        } else {
            for (int m = 0; m < maxNumPred; ++m) {
                int i = predecessors[m][idx][0];
                int weight = predecessors[m][idx][1];
                if (i == -1) {
                    break;
                }
                if (fixed[i] && (in_latticeValues[idx] >= in_latticeValues[i] + weight)) {
                    fixed[idx] = 1;
                }
            }

            if (fixed[idx]) {
                return;
            }

            int newMin = -1;
            for (int m = 0; m < maxNumPred; ++m) {
                int i = predecessors[m][idx][0];
                int weight = predecessors[m][idx][1];
                if (i == -1) {
                    break;
                }
                int candidate = in_latticeValues[i] + weight;
                if (newMin == -1 || (candidate < newMin)) {
                    newMin = candidate;
                }
            }

            out_latticeValues[idx] = max(beta, newMin);
        }
    }
}

void llpFindMin(int idx, int (*predecessors) [N][2], int maxNumPred, int *latticeValues, int *fixed, int *vals, int *nonempty) {
    if (idx < N) {
        int minVal = -1;
        int found = 0;
        for (int i = 0; i < maxNumPred; i += 1) {
            if (predecessors[i][idx][0] == -1) {
                break;
            }
            int pred = fixed[predecessors[i][idx][0]] && !(fixed[idx]);
            if (pred) {
                found = 1;
            } else {
                continue;
            }
            int candidate = latticeValues[predecessors[i][idx][0]] + predecessors[i][idx][1];
            if (minVal == -1 || (candidate < minVal)) {
                minVal = candidate;
            }
        }
        vals[idx] = minVal;
        if (found) {
            *nonempty = 1;
        }
    }
}

void minReduce(int idx, int *in_vals, int *out_vals, int s) {
    if (idx % (2*s) != 0) {
        return;
    }

    if (in_vals[idx] == -1) {
        out_vals[idx] = in_vals[idx+s];
    } else if (in_vals[idx+s] == -1) {
        out_vals[idx] = in_vals[idx];
    } else {
        out_vals[idx] = min(in_vals[idx], in_vals[idx+s]);
    }
}


int main() {
    int error = 0;

    /* https://stackoverflow.com/questions/2150291/how-do-i-measure-a-time-interval-in-c */
    struct timeval t1, t2;
    double elapsedTime;

    /* seed -- create random graph */
    srand(128);

    int (*h_predecessors)[N][2];
    int predBytes = NUM_PRED * sizeof *h_predecessors;
    h_predecessors = (int (*)[N][2])malloc(predBytes);

    /* initialize random graph (plus a linear path beginning to end) */
    for (int i = 0; i < N; ++i) {
        h_predecessors[0][i][0] = i-1;
        h_predecessors[0][i][1] = 1;


        int j = 1;
        while (j < min(N-1,NUM_PRED)) {
            int pred = rand() % N;
            int weight = (MAXWEIGHT > 1) ? (1 + (rand() % (MAXWEIGHT - 1))) : 1;

            // no self-loop
            if (pred == i) {
                continue;
            }

            // check it was not already used
            int found = 0;
            for (int k = 0; k < j; ++k) {
                if (h_predecessors[k][i][0] == pred) {
                    found = 1;
                }
            }
            if (found) {
                continue;
            }

            h_predecessors[j][i][0] = pred;
            h_predecessors[j][i][1] = weight;
            ++j;
        }

        // sentinel
        if (j < NUM_PRED) {
            h_predecessors[j][i][0] = -1;
        }
    }


    int *h_latticeValues;
    int *h_latticeValues_2;
    int bytes = N * sizeof *h_latticeValues;
    h_latticeValues = (int*)malloc(bytes);
    h_latticeValues_2 = (int*)malloc(bytes);
    for (int i = 0; i < N; ++i) {
        h_latticeValues[i] = 0;
        h_latticeValues_2[i] = 0;
    }

    int *h_fixed;
    h_fixed = (int*)malloc(bytes);
    h_fixed[0] = 1;
    for (int i = 1; i < N; ++i) {
        h_fixed[i] = 0;
    }

    int *h_vals;
    int *h_vals_2;
    h_vals = (int*)malloc(bytes);
    h_vals_2 = (int*)malloc(bytes);
    for (int i = 0; i < N; ++i) {
        h_vals[i] = 0;
        h_vals_2[i] = 0;
    }

    int *h_ret;
    int *h_nonempty;
    h_ret = (int*)malloc(1);
    *h_ret = 0;
    h_nonempty = (int*)malloc(1);
    *h_nonempty = 0;

    int done = 0;

    gettimeofday(&t1, NULL);

    while(1) {
        *h_ret = 0;
        *h_nonempty = 0;

        for (int idx = 0; idx < N; ++idx) {
            hasUnfixed(idx, h_ret, h_fixed);
        }


        if (*h_ret == 0) {
            done = 1;
            break; /* we are done */
        }



        for (int idx = 0; idx < N; ++idx) {
            llpFindMin(idx, h_predecessors, NUM_PRED, h_latticeValues, h_fixed, h_vals, h_nonempty);
        }
        if (*h_nonempty == 0) {
            printf("------ ERROR: non-fixed nodes not reachable ------");
            error = 1;
            break;
        }

        memcpy(h_vals_2, h_vals, bytes);

        for (int s = 1; s < N; s *= 2) {
            for (int idx = 0; idx < N; ++idx) {
                minReduce(idx, h_vals_2, h_vals_2, s);
            }
        }

        int beta = h_vals_2[0];

        for (int idx = 0; idx < N; ++idx) {
            llpThreadKernel(idx, h_predecessors, NUM_PRED, h_latticeValues, h_latticeValues_2, h_fixed, h_vals, beta);
        }
        memcpy(h_latticeValues, h_latticeValues_2, bytes);
    }

    gettimeofday(&t2, NULL);

    printf("\n");

    if (done && !error) {
        printf("DONE\n");
        printf("------ BEGIN VALUES ------\n");
        for (int i = 0; i < N; ++i) {
            printf("%d: %d\n", i, h_latticeValues[i]);
        }
        elapsedTime = (t2.tv_sec - t1.tv_sec) * 1000.0;
        elapsedTime += (t2.tv_usec - t1.tv_usec) / 1000.0;
        printf("------ END VALUES ------\n");
        printf("Time %f ms.\n", elapsedTime);
    } else {
        printf("ERROR\n");
    }

    free(h_predecessors);
    free(h_latticeValues);
    free(h_latticeValues_2);
    free(h_fixed);
    free(h_vals);
    free(h_vals_2);
    free(h_ret);
    free(h_nonempty);

}


'File written in /content/src/cpu_SSSP_LLP.cu'

In [13]:
!nvcc -arch=sm_75 -o "/content/src/cpu_SSSP_LLP.o" /content/src/cpu_SSSP_LLP.cu

In [14]:
!chmod 755 /content/src/cpu_SSSP_LLP.o
!/content/src/cpu_SSSP_LLP.o

Streaming output truncated to the last 5000 lines.
27770: 13
27771: 13
27772: 14
27773: 14
27774: 15
27775: 14
27776: 14
27777: 7
27778: 8
27779: 9
27780: 10
27781: 11
27782: 12
27783: 13
27784: 12
27785: 11
27786: 12
27787: 13
27788: 11
27789: 12
27790: 13
27791: 13
27792: 10
27793: 11
27794: 12
27795: 13
27796: 13
27797: 12
27798: 13
27799: 14
27800: 14
27801: 10
27802: 10
27803: 10
27804: 11
27805: 12
27806: 13
27807: 12
27808: 13
27809: 14
27810: 12
27811: 10
27812: 11
27813: 8
27814: 9
27815: 10
27816: 11
27817: 9
27818: 10
27819: 11
27820: 12
27821: 7
27822: 8
27823: 9
27824: 10
27825: 11
27826: 12
27827: 13
27828: 13
27829: 12
27830: 10
27831: 11
27832: 10
27833: 8
27834: 9
27835: 10
27836: 10
27837: 11
27838: 12
27839: 12
27840: 13
27841: 14
27842: 14
27843: 14
27844: 12
27845: 13
27846: 11
27847: 12
27848: 13
27849: 12
27850: 13
27851: 12
27852: 13
27853: 13
27854: 14
27855: 8
27856: 9
27857: 10
27858: 11
27859: 12
27860: 13
27861: 12
27862: 10
27863: 11
27864: 12
27865: 13
27